# [과제]yes24 도서 웹 크롤링
1. 책 키워드 입력해서 url 구성

2. 스크래핑 내용
- 제목, 저자, 출판사, 출판일, 가격, 평점(정적 or 동적)
- 페이지 이동해서 계속해서 반복 스크래핑(동적)
- 이미지 다운로드

3. DataFrame으로 작성해서 csv 파일로 저장

4. mysql DB에 저장

In [2]:
import urllib.request as req1
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pymysql

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [3]:
urld="http://www.yes24.com/Product/Search?domain=ALL&query=%EC%8B%9D%EB%AC%BC&page=1"
driver=webdriver.Chrome('chromedriver')
driver.implicitly_wait(5)
driver.get(urld)
time.sleep(2)

In [4]:
def image_save(res,i):
    save_name='data/book_images/image'+str(i)+'.jpg'
    req1.urlretrieve(res,save_name)
    print(save_name,'가 저장됨')

In [5]:
url="http://www.yes24.com/Product/Search?domain=ALL&query=%EC%8B%9D%EB%AC%BC&page="
j=1

book_list=[]
i=0
stopFlag=False
flag=True

for n in range(2,11):
    res=req1.urlopen(url+str(j))
    bs=BeautifulSoup(res,'html.parser')
    books=bs.select('#yesSchList > li')

    for book in books:
        title=book.select('div > div.item_info > div.info_row.info_name > a.gd_name')
        author=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_auth')
        publisher=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_pub > a')
        date=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_date')
        price=book.select('div > div.item_info > div.info_row.info_price > strong > em')
        rating=book.select('div > div.item_info > div.info_row.info_rating > span.rating_grade > em')

        if title == []:
            title='X'
        else:
            title=title[0].text

        if author == []:
            author='X'
        else:
            author=author[0].text.replace("\n","").replace("\r","").replace("  ","")

        if publisher == []:
            publisher='X'
        else:
            publisher=publisher[0].text

        if date == []:
            date='X'
        else:
            date=date[0].text

        if price == []:
            price='X'
        else:
            price=price[0].text

        if rating == []:
            rating='X'
        else:
            rating=rating[0].text

        b_list=(title,author,publisher,date,price,rating)
        print(b_list)

        # 상품 이미지 저장
        i+=1
        img=book.select('div > div.item_img > div.img_canvas > span > span > a > em > img')
        img_src=img[0]['data-original']
        image_save(img_src,i)

        # book_list에 추가
        book_list.append(b_list)
        
        time.sleep(1)

    book_count=len(driver.find_elements(By.CSS_SELECTOR, '#yesSchList > li'))
    j+=1
    if book_count<24:
        driver.get(url)
        driver.quit()
        flag=False
    else:
        linkurl='#goodsListWrap > div.sGoodsPagen > div > a:nth-child(%d)'%n

        if bs.select(linkurl)!=[]:
            linkNum=driver.find_element(By.CSS_SELECTOR, linkurl)
            linkNum.click()
            time.sleep(2)
        else:
            driver.get(url)
            driver.quit()
            # flag=False

if flag!=False:
    nextblock='#goodsListWrap > div.sGoodsPagen > div > a.bgYUI.next'
    nextNum=driver.find_element(By.CSS_SELECTOR,nextblock)
    nextNum.click()
    time.sleep(2)

while flag:
    for n in range(4,13):
        res=req1.urlopen(url+str(j))
        bs=BeautifulSoup(res,'html.parser')
        books=bs.select('#yesSchList > li')
        for book in books:
            title=book.select('div > div.item_info > div.info_row.info_name > a.gd_name')
            author=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_auth')
            publisher=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_pub > a')
            date=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_date')
            price=book.select('div > div.item_info > div.info_row.info_price > strong > em')
            rating=book.select('div > div.item_info > div.info_row.info_rating > span.rating_grade > em')
            
                        
            if title == []:
                title='X'
            else:
                title=title[0].text
                
            if author == []:
                author='X'
            else:
                author=author[0].text.replace("\n","").replace("\r","").replace("  ","")
                
            if publisher == []:
                publisher='X'
            else:
                publisher=publisher[0].text
                
            if date == []:
                date='X'
            else:
                date=date[0].text
                
            if price == []:
                price='X'
            else:
                price=price[0].text
                
            if rating == []:
                rating='X'
            else:
                rating=rating[0].text
                
            b_list=(title,author,publisher,date,price,rating)
            
            # 상품 이미지 저장
            i+=1
            img=book.select('div > div.item_img > div.img_canvas > span > span > a > em > img')
            img_src=img[0]['data-original']
            image_save(img_src,i)

            # book_list에 추가
            book_list.append(b_list)
            print(b_list)
            
            time.sleep(1)

        book_count=len(driver.find_elements(By.CSS_SELECTOR, '#yesSchList > li'))
        j+=1
        
        linkurl='#goodsListWrap > div.sGoodsPagen > div > a:nth-child(%d)'%n
        linkNum=driver.find_element(By.CSS_SELECTOR, linkurl)
        linkNum.click()
        time.sleep(2)
    
        if book_count<24:
            stopFlag=True
            break
    if stopFlag==True:
        break
        
    nextblock='#goodsListWrap > div.sGoodsPagen > div > a.bgYUI.next'
    nextNum=driver.find_element(By.CSS_SELECTOR,nextblock)
    nextNum.click()
    time.sleep(2)
    
    # 다음 블록 첫 번째 페이지
    res=req1.urlopen(url+str(j))
    bs=BeautifulSoup(res,'html.parser')
    books=bs.select('#yesSchList > li')
    
    for book in books:
        title=book.select('div > div.item_info > div.info_row.info_name > a.gd_name')
        author=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_auth')
        publisher=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_pub > a')
        date=book.select('div > div.item_info > div.info_row.info_pubGrp > span.authPub.info_date')
        price=book.select('div > div.item_info > div.info_row.info_price > strong > em')
        rating=book.select('div > div.item_info > div.info_row.info_rating > span.rating_grade > em')

        if title == []:
            title='X'
        else:
            title=title[0].text

        if author == []:
            author='X'
        else:
            author=author[0].text.replace("\n","").replace("\r","").replace("  ","")

        if publisher == []:
            publisher='X'
        else:
            publisher=publisher[0].text

        if date == []:
            date='X'
        else:
            date=date[0].text

        if price == []:
            price='X'
        else:
            price=price[0].text

        if rating == []:
            rating='X'
        else:
            rating=rating[0].text

        b_list=(title,author,publisher,date,price,rating)
        print(b_list)
            
        # book_list에 추가
        book_list.append(b_list)

        # 상품 이미지 저장
        i+=1
        img=book.select('div > div.item_img > div.img_canvas > span > span > a > em > img')
        img_src=img[0]['data-original']
        image_save(img_src,i)
        
        time.sleep(1)

    book_count=len(driver.find_elements(By.CSS_SELECTOR, '#yesSchList > li'))
    j+=1
    if book_count<24:
        stopFlag=True
        break
        
    linkurl='#goodsListWrap > div.sGoodsPagen > div > a:nth-child(4)'
    linkNum=driver.find_element(By.CSS_SELECTOR, linkurl)
    linkNum.click()
    time.sleep(2)

if flag!=False:
    driver.get(url)
    driver.quit()

('이웃집 식물상담소', '신혜우 저', '브라이트', '2022년 05월', '15,300', '10.0')
data/book_images/image1.jpg 가 저장됨
('가꾸는 이의 즐거움', '이유리 저/오리여인 그림', '예스이십사', '2022년 04월', '0', '9.8')
data/book_images/image2.jpg 가 저장됨
('나라는 식물을 키워보기로 했다', '김은주 글/워리 라인스 그림', '허밍버드', '2021년 07월', '13,500', '9.7')
data/book_images/image3.jpg 가 저장됨
('식물학자의 노트', '신혜우 글그림', '김영사', '2021년 04월', '17,820', '9.8')
data/book_images/image4.jpg 가 저장됨
('알찬 중등 과학 2-1 4단원 (2022년용)', '편집부 편', '비상ESN', '2020년 02월', '1,800', 'X')
data/book_images/image5.jpg 가 저장됨
('글로스터의 홈가드닝 이야기 ', '글로스터 저/아피스토 그림', '미디어샘', '2022년 06월', '17,820', 'X')
data/book_images/image6.jpg 가 저장됨
('식물이 아프면 찾아오세요', '김강호 저', '길벗', '2020년 12월', '15,750', '9.8')
data/book_images/image7.jpg 가 저장됨
('식물 상담', '강세종 저', '북하우스', '2021년 11월', '16,200', '9.8')
data/book_images/image8.jpg 가 저장됨
('마음을 그리면 꽃', '전유리 저', '클', '2022년 05월', '13,500', '9.7')
data/book_images/image9.jpg 가 저장됨
('도시 식물 탐험대 ', '손연주, 박민지, 안현지 글/김완순 감수', '주니어RHK', '2022년 05월', '9,900', 'X')
data/book_images/image

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#goodsListWrap > div.sGoodsPagen > div > a:nth-child(4)"}
  (Session info: chrome=102.0.5005.63)
Stacktrace:
Backtrace:
	Ordinal0 [0x00DBD953+2414931]
	Ordinal0 [0x00D4F5E1+1963489]
	Ordinal0 [0x00C3C6B8+837304]
	Ordinal0 [0x00C69500+1021184]
	Ordinal0 [0x00C6979B+1021851]
	Ordinal0 [0x00C96502+1205506]
	Ordinal0 [0x00C844E4+1131748]
	Ordinal0 [0x00C94812+1198098]
	Ordinal0 [0x00C842B6+1131190]
	Ordinal0 [0x00C5E860+976992]
	Ordinal0 [0x00C5F756+980822]
	GetHandleVerifier [0x0102CC62+2510274]
	GetHandleVerifier [0x0101F760+2455744]
	GetHandleVerifier [0x00E4EABA+551962]
	GetHandleVerifier [0x00E4D916+547446]
	Ordinal0 [0x00D55F3B+1990459]
	Ordinal0 [0x00D5A898+2009240]
	Ordinal0 [0x00D5A985+2009477]
	Ordinal0 [0x00D63AD1+2046673]
	BaseThreadInitThunk [0x7714FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77CE7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77CE7A4E+238]


In [3]:
# DataFrame 작성
df=pd.DataFrame(book_list,columns=['title','author','publisher','date','price','rating'])
# df.to_csv('data/books.csv', index=False)
df

NameError: name 'book_list' is not defined

In [82]:
# mysql에 데이터 저장
conn=pymysql.connect(host='localhost',user='lyk',password='1234',db='pydb',charset='utf8')
cursor=conn.cursor()
sql="insert into yes_book_practice(title,author,publisher,date,price,rating) values(%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql,book_list)
conn.commit()
conn.close()